In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader

import pickle, os, sys, json
from tqdm import tqdm 
# imagebind
from models.venezia import load_venezia_pretrain, Venezia
from utils.model_control import freeze_model, run_train_test_2_input
from utils.custom_logger import get_logger
from dataset import dataset

c:\Users\esill\miniconda3\envs\imagebind\lib\site-packages\torchaudio\backend\utils.py:62: UserWarning: No audio backend is available.
  warnings.warn("No audio backend is available.")
c:\Users\esill\miniconda3\envs\imagebind\lib\site-packages\torchvision\transforms\_functional_video.py:6: UserWarning: The 'torchvision.transforms._functional_video' module is deprecated since 0.12 and will be removed in the future. Please use the 'torchvision.transforms.functional' module instead.
  warnings.warn(
c:\Users\esill\miniconda3\envs\imagebind\lib\site-packages\torchvision\transforms\_transforms_video.py:22: UserWarning: The 'torchvision.transforms._transforms_video' module is deprecated since 0.12 and will be removed in the future. Please use the 'torchvision.transforms' module instead.
  warnings.warn(


In [3]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"

In [4]:
touch = 'softness'
train = torch.load(f'./dataset/{touch}_train.pth')
test = torch.load(f'./dataset/{touch}_test.pth')

In [5]:
train_loader = DataLoader(train, batch_size=64, shuffle=True)
test_loader = DataLoader(test, batch_size=32, shuffle=False)

In [6]:
#model = load_venezia_pretrain()
model = Venezia()
model.load_state_dict(torch.load(f'.checkpoints/best/venezia_best.pth'))
freeze_model(model, exception=
             ['classifier', 'polling', 'input_preprocessor']) # modality_head

model.to(device)

c:\Users\esill\miniconda3\envs\imagebind\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\esill\miniconda3\envs\imagebind\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V3_Small_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V3_Small_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


 polling weight True
 polling bias True
classifier_ input_preprocessor fc.0.weight True
classifier_ input_preprocessor fc.0.bias True
classifier_ input_preprocessor processor.0.weight True
classifier_ input_preprocessor processor.0.bias True
classifier_ input_preprocessor processor.1.weight True
classifier_ input_preprocessor processor.1.bias True
mobilenet classifier 0.weight True
mobilenet classifier 0.bias True
mobilenet classifier 3.weight True
mobilenet classifier 3.bias True


Venezia(
  (image_encoder): CustomIbvisEncoder(
    (modality_preprocessors): RGBDTPreprocessor(
      (cls_token): tensor((1, 1, 1280), requires_grad=False)
      
      (rgbt_stem): PatchEmbedGeneric(
        (proj): Sequential(
          (0): PadIm2Video()
          (1): Conv3d(3, 1280, kernel_size=(2, 14, 14), stride=(2, 14, 14), bias=False)
        )
      )
      (pos_embedding_helper): SpatioTemporalPosEmbeddingHelper(
        (pos_embed): tensor((1, 257, 1280), requires_grad=False)
        
      )
    )
    (modality_trunks): SimpleTransformer(
      (pre_transformer_layer): Sequential(
        (0): LayerNorm((1280,), eps=1e-06, elementwise_affine=True)
        (1): EinOpsRearrange()
      )
      (blocks): Sequential(
        (0): BlockWithMasking(
          (attn): MultiheadAttention(
            (out_proj): NonDynamicallyQuantizableLinear(in_features=1280, out_features=1280, bias=True)
          )
          (drop_path): Identity()
          (norm_1): LayerNorm((1280,), eps=

In [7]:
# 손실 함수 및 최적화 함수 정의
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
num_epochs = 5
test_step = 1
logger = get_logger('venezia_stable')

In [8]:
logger.info('start_new_run--------------------------------')

result = run_train_test_2_input(
    model=model, 
    train_loader=train_loader, 
    test_loader=test_loader,
    best_test_loss = float('inf'),
    criterion=criterion, 
    optimizer=optimizer, 
    num_epochs=num_epochs, 
    test_step=test_step, 
    logger = logger, 
    device = device
    path = '.checkpoints/best/venezia_best.pth')

batch: 1/ epoch: 0: 100%|██████████| 37/37 [3:49:42<00:00, 372.50s/it]  
